# 4word AI

The purpose of this repo is to generate puzzles for the 4 word game.  The challenge is estimating how a difficult a puzzle is given that we lack real user data.  When we dont have data, machine learning cant work, so we have to rely on math to generate smart heuristics.

## Approach

We want to use the minimum possible solve path as the main metric for the difficulty. This almost does well enough on its own, accept for 2 caveats.

1. This apporach does not account for the number of paths from a to b assume a puzzle is solved in only a few steps, but it there is only one way to do it.  If the player does not know this word, they will be forced to guess, or be stuck
2. We dont account for word frequency.  i.e. say a game is solvable in a short amount of steps, but it requires the user to know a very obscure word.  In this case the puzzle should be more difficult

## Get words sorted by frequency

Knowing how frequently the words are used will serve usefull when considering how difficult a game is

Data source: https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists

let $U_{l}$ be the universe of all words of length $l$

In [1]:
from src.build4WordUniverse import processRawWikiDataIntoPaginatedData

U=processRawWikiDataIntoPaginatedData()
U

,word,count
0,THAT,413389
1,WHAT,285826
2,THIS,249860
3,KNOW,241548
4,HAVE,210523
...,...,...
3200,APPS,6
3201,ANKA,6
3202,ANDA,6
3203,AMAH,6


We notice that we have only 3200 words of length 4 form the data that we collected.  Any words discovered later will be appended to the back, with a count of 0.  This data is saved in `data/4words/universe.csv` and `data/4words/univrse.json` for easy retrival.


Before we start going right for the gola, we will stop and think what data structures emerge.  

The first the comes to mind is a graph where each node is a word, and the edges are present if we can change 1 letter of the word to get from one to the other.  Note, this relationship is symetric.  Using this model, we can see a game is possible, $\iff$ there exists a path.

Thee first quantitty we want to calculate is a boolean matrix that represents if we can get from word a to word b in 1 move.  We will use this as a helper to do quick look ups.  The runtime for calculating this matrix is $O(D^2)$

